## Agent Framework Context Providers


#### Introduction to Azure AI Search

**Azure AI Search** is a fully managed search-as-a-service that lets you add sophisticated search capabilities to your apps without having to manage the whole search infrastructure.

- **Core capabilities:** Azure AI Search indexes your content (documents, databases, files, etc.) and provides fast, relevant search results with features like full-text search, filtering, faceting, autocomplete, and semantic search. The "AI" part comes from its integration with cognitive services for things like Optical Character Recognition (OCR), key phrase extraction, entity recognition, and language detection during indexing.
- **Data sources:** You can pull data from various sources like Azure Blob Storage, Azure SQL Database, Cosmos DB, or push data directly via REST APIs. It supports many file formats including PDFs, Office documents, HTML, JSON, and plain text.

Beyond basic keyword search, Azure AI Search offers semantic search (understanding meaning and context), vector search for similarity matching, hybrid search combining multiple approaches, and AI enrichment to extract insights from unstructured content. You can also customize ranking, add synonyms, and support multiple languages.

##### Why are we using Azure AI Search as a context provider?

It will do everything for us! 

Chat models face constraints on the amount of data they can accept on a request. You should use Azure AI Search because the quality of content passed to an LLM can make or break a RAG solution.


#### Example: Create a RAG Agent
In this section, you will learn how to build a retrieval-augmented generation (RAG) agent with Microsoft Agent Framework. 

First, go to the notebook '03.0-setup-guide.ipynb' for a step-by-step guide on how to deploy all the necessary resources to create the Azure AI Search Index.


Now that we've vectorized our documents and created an index, we can build a Search Agent with Microsoft Agent Framework. This agent will leverage the Azure AI Search index to retrieve relevant information from your documents in response to user queries. In this section, you'll:

- Connect your code to the Azure AI Search index you created earlier
- Define and configure a search agent with the appropriate tools and resources
- Interact with the agent to perform intelligent, context-aware document retrieval

By the end of this lab, you’ll understand the Retrieval-Augmented Generation (RAG) pattern and how to implement it using Azure AI services for multi-agent scenarios.

For the following exercise, make sure to add the following environment variables to your .env file:

```python
AZURE_SEARCH_ENDPOINT=https://<search-service-name>.search.windows.net
INDEX_NAME=<index-name>
SEARCH_API_KEY=<search-service-key>
```

In [1]:
import os
import json
from dotenv import load_dotenv
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()

# Your existing configuration
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# New credentials we've added to enable search
search_key = os.getenv("SEARCH_API_KEY")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = os.getenv("INDEX_NAME")

You’re creating two clients:

- AzureOpenAIChatClient connects to your Azure OpenAI deployment using the endpoint, API key, version, and deployment name so you can send prompts to the model.
- SearchClient connects to your Azure AI Search index using its endpoint, index name and key, allowing you to retrieve relevant documents.

Together these create a RAG workflow: fetch context from search, then pass it to the model for grounded answers.

In [2]:
# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

# Initialize search client with correct credentials
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

The `AzureAISearchTool` wrapper that turns your SearchClient into a callable “tool” for an agent. It’s initialized with an existing Azure AI Search SearchClient. 

The search(query, top) method runs a semantic search against your index, collects each hit’s chunk content and @search.score, and returns them as a JSON-formatted string for easy injection into a prompt. If Azure Search returns an error (HttpResponseError), it prints basic diagnostics and returns a JSON error payload instead:

In [3]:
class AzureAISearchTool:
    """Function tool to search Azure AI Search index"""
   
    def __init__(self, search_client: SearchClient):
        self.search_client = search_client
   
    def search(self, query: str, top: int = 5) -> str:
        """
        Search the Azure AI Search index for relevant documents
       
        Args:
            query: The search query about lighting technology advancements or related topics
            top: Number of results to return (default 5)
       
        Returns:
            JSON string containing the search results with chunks and metadata
        """
        try:
            results = self.search_client.search(
                query,
                top=top,
                query_type="semantic"
            )
           
            documents = []
            for doc in results:
                documents.append({
                    "chunk": doc.get('chunk', ''),
                    "score": doc.get('@search.score', 0)
                })
           
            # Return formatted results
            return json.dumps(documents, indent=2)
           
        except HttpResponseError as e:
            print("HTTP status:", e.status_code if hasattr(e, "status_code") else "N/A")
            if e.response is not None:
                try:
                    print("Response text:", e.response.text())
                except Exception:
                    print("Could not get response.text()")
            return json.dumps({"error": str(e)})

Finally, we create a simple agent and run it with a user question. The agent uses your Azure OpenAI model for reasoning and the Azure AI Search tool for retrieval. It follows the given instructions to stay factual and cite sources. 

When you run the cell, you should see a grounded response based on the indexed documents:

In [4]:
# Initialize the search tool for the agent providing the search client
tools = AzureAISearchTool(search_client=search_client)

faq_agent = client.create_agent(
    name="employee_faq_agent",
    instructions="""
    You are a helpful HR assistant for Contoso Electronics employees. You answer questions about:
    - Employee benefits and plan packages
    - Company policies and employee handbook guidelines
    - Health and wellness reimbursement programs
    - Role descriptions and responsibilities
    
    Be friendly, professional, and concise. Answer ONLY using facts from the retrieved documents. 
    If information is not in the sources, clearly state: "I don't have that information in our current documentation. Please contact HR at [HR contact] for assistance."
    Never make up or infer information beyond what's explicitly stated
    
    Keep answers brief but complete. If a question is ambiguous, ask a clarifying question before searching.
    For sensitive topics (disciplinary actions, terminations, complaints), direct employees to contact HR directly.
    For benefits enrollment changes, mention that actions may require contacting benefits administration.

    """,
    
    tools=[tools.search]
)

# Minimal run sample with a single question
user_question = "Can you tell me what's not included in PerksPlus?"
result = await faq_agent.run(
    user_question,
    )

print(result)

PerksPlus does not cover the following items:

- Non-fitness related expenses
- Medical treatments and procedures
- Travel expenses (unless related to a fitness program)
- Food and supplements

If you have further questions about the program, feel free to ask!


#### Example - Handoff orchestration with contextualized RAG

In the following example, we'll use the created RAG tool to create a simple version of an employee support system capable of giving specialized context on multiple data sources. To showcase agent capabilities on a use case, we will use a simple dataset with existing employee data. In real life, that would be an actual employee DB and would require more sophisticated querying.

##### TODO - Handoff Explanation

First, let's set up a mock employee information database which the agent can refer to when providing support on employee questions about their healthcare plans and options.

In [5]:
# Mock employee database for Contoso Electronics
employees = [
    {
        "employee_id": 1001,
        "first_name": "Sarah",
        "last_name": "Chen",
        "email": "sarah.chen@contoso.com",
        "role": "Senior Software Engineer",
        "department": "Engineering",
        "health_plan": "Northwind Health Plus",
        "dental_coverage": False,
        "vision_coverage": True,
        "perks_plus_spent": 847.50,
        "last_review_date": "2024-10-15"
    },
    {
        "employee_id": 1002,
        "first_name": "Michael",
        "last_name": "Rodriguez",
        "email": "michael.rodriguez@contoso.com",
        "role": "Product Manager",
        "department": "Product",
        "health_plan": "Northwind Health Plus",
        "dental_coverage": True,
        "vision_coverage": True,
        "perks_plus_spent": 623.00,
        "last_review_date": "2024-11-20"
    },
    {
        "employee_id": 1003,
        "first_name": "Emily",
        "last_name": "Thompson",
        "email": "emily.thompson@contoso.com",
        "role": "Marketing Specialist",
        "department": "Marketing",
        "health_plan": "Northwind Standard",
        "dental_coverage": True,
        "vision_coverage": False,
        "perks_plus_spent": 245.75,
        "last_review_date": "2024-09-12"
    },
    {
        "employee_id": 1005,
        "first_name": "David",
        "last_name": "Kumar",
        "email": "david.kumar@contoso.com",
        "role": "Sales Representative",
        "department": "Sales",
        "health_plan": "Northwind Standard",
        "dental_coverage": False,
        "vision_coverage": False,
        "perks_plus_spent": 125.00,
        "last_review_date": "2024-11-15"
    }
]

Then we need to set up tools for the agents to work with. For the minimal sample of the employee support system, we'll have the following ones:

In [6]:
# Tools for the Database agent

def get_user() -> str:
    """Get the id of the current user."""
    print("executing get_user")
    return "1002"


def get_employee_info(employee_id: int) -> str:
    """Look up employee information by employee ID."""
    print("executing get_employee_info")
    print(f"employee_id: {employee_id}")
    
    employee = next((emp for emp in employees if emp["employee_id"] == employee_id), None)
    return json.dumps(employee, indent=2)


def get_health_plan_info(employee_id: int) -> str:
    """Get health plan and coverage details for an employee."""
    print("executing get_health_plan_info")
    print(f"employee_id: {employee_id}")
    
    employee = next((emp for emp in employees if emp["employee_id"] == employee_id), None)
    
    plan_info = {
        "employee": f"{employee['first_name']} {employee['last_name']}",
        "health_plan": employee["health_plan"],
        "dental_coverage": employee["dental_coverage"],
        "vision_coverage": employee["vision_coverage"],
        "last_review_date": employee["last_review_date"]
    }
    return json.dumps(plan_info, indent=2)

    
def get_perks_info(employee_id: int) -> str:
    """Get PerksPlus spending information for an employee."""
    print("executing get_perks_info")
    print(f"employee_id: {employee_id}")
    
    employee = next((emp for emp in employees if emp["employee_id"] == employee_id), None)
    
    perks_info = {
        "employee": f"{employee['first_name']} {employee['last_name']}",
        "perks_plus_spent": employee["perks_plus_spent"],
        "perks_plus_remaining": 1000.00 - employee["perks_plus_spent"]
    }
    return json.dumps(perks_info, indent=2)

In [8]:
import logging
from typing import cast

from agent_framework import (
    AgentRunResponseUpdate,
    AgentRunUpdateEvent,
    ChatAgent,
    ChatMessage,
    HandoffBuilder,
    WorkflowEvent,
    WorkflowOutputEvent,
)

logging.basicConfig(level=logging.ERROR)


last_response_id: str | None = None


def _display_event(event: WorkflowEvent) -> None:
    """Print the final conversation snapshot from workflow output events."""
    if isinstance(event, AgentRunUpdateEvent) and event.data:
        update: AgentRunResponseUpdate = event.data
        if not update.text:
            return
        global last_response_id
        if update.response_id != last_response_id:
            last_response_id = update.response_id
            print(f"\n- {update.author_name}: ", flush=True, end="")
        print(event.data, flush=True, end="")
    elif isinstance(event, WorkflowOutputEvent):
        conversation = cast(list[ChatMessage], event.data)
        print("\n=== Final Conversation (Autonomous with Iteration) ===")
        for message in conversation:
            speaker = message.author_name or message.role.value
            text_preview = message.text[:200] + "..." if len(message.text) > 200 else message.text
            print(f"- {speaker}: {text_preview}")
        print(f"\nTotal messages: {len(conversation)}")
        print("=====================================================")


db_agent = client.create_agent(
    name="db_agent",
    instructions="""
    You retrieve and answer questions about employee data for Contoso Electronics.
    
    1. Call get_user() then retrieve the requested data
    2. If the question is purely about data (balance, dates, plan name, coverage status), answer it completely
    """,
    tools=[get_user, get_employee_info, get_health_plan_info, get_perks_info]
)


faq_agent = client.create_agent(
    name="faq_agent",
    instructions="""
    You explain benefits and policies for Contoso Electronics using the knowledge base.
    
    1. Check conversation history for employee context from db_agent
    2. Search knowledge base for relevant information
    3. Provide personalized explanation using their context
    4. Suggest coverage additions if they have gaps, or PerksPlus uses if budget remains
    
    Cite sources as [Source: Document Name]. Never fabricate information.
    """,
    tools=[tools.search]
)

triage_agent = client.create_agent(
    instructions="""
    You coordinate employee benefits support for Contoso Electronics.
   
    ROUTING LOGIC:
   
    1. **Data-only questions** ("what's my balance", "what plan do I have", "when does my coverage start"):
       - Transfer to db_agent
       - When db_agent returns, provide the answer to user
       - Ask: "Is there anything else I can help you with?"
   
    2. **Explanation questions needing context** ("what's included in MY plan", "what can I use MY perks for", "possibilities in MY healthcare plan"):
       - Transfer to db_agent FIRST to get employee data
       - When db_agent returns, transfer to faq_agent with instruction: "Using the employee data above, explain [original question]"
       - When faq_agent returns, synthesize both responses into complete answer
       - Ask: "Is there anything else I can help you with?"
   
    3. **General policy questions** ("what is PerksPlus", "how does dental coverage work", "what's the difference between plans"):
       - Transfer to faq_agent directly
       - When faq_agent returns, provide explanation to user
       - Ask: "Is there anything else I can help you with?"
   
    CRITICAL: After each handoff completes, YOU must continue orchestrating. Don't wait for agents to transfer back.
    Monitor the conversation and route to the next agent when needed.
    """,
    name="triage_agent"
    )

workflow = (
        HandoffBuilder(
            name="employee_support",
            participants=[triage_agent, faq_agent, db_agent],
        )
        .set_coordinator(triage_agent)
        .add_handoff(triage_agent, [faq_agent, db_agent])
        .add_handoff(db_agent, [triage_agent])              # DB returns to triage
        .add_handoff(faq_agent, [triage_agent])             # FAQ returns to triage
        .build()
    )

request = "What are the healthplans offered?"
print("Request:", request)
async for event in workflow.run_stream(request):
     _display_event(event)


Request: What are the healthplans offered?

- triage_agent: I'm currently waiting for a response from the FAQ agent regarding the health plans offered at Contoso Electronics. Please hold on for a moment while I get the information.
- faq_agent: Contoso Electronics offers two health plans through Northwind Health:

1. **Northwind Health Plus**:
   - **Comprehensive Coverage**: This plan provides extensive coverage for medical, vision, and dental services. It includes prescription drug coverage, mental health and substance abuse coverage, and preventive care services.
   - **Provider Options**: Employees can choose from a wide range of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies.
   - **Emergency Services**: Coverage for emergency services is available, including both in-network and out-of-network options.
   - **Additional Benefits**: This plan also covers routine physicals, immunizations, cancer screenings, and hospital stays.

2. **N

### TODO - Exercise
Let's build up on our handoff example. We'll add another agent that is connected to a second knowledge source that provides in-depth guidance on the two healthcare plans available to employees. You'll need to create another index in the same way we created the first one and extend the handoff system 



*Note*: AF now supports `AzureAISearchContextProvider` that automatically grounds agent responses with Azure AI Search data. This context provider lets an agent retrieve relevant context from Azure AI Search without you manually wiring a search tool each time:


```python
from agent_framework.azure import AzureAISearchContextProvider

search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=api_key,  # Use api_key for API key auth, or credential for managed identity
    credential=AzureKeyCredential(search_key),
    mode="semantic",  # Default mode
    top_k=3,  # Retrieve top 3 most relevant documents
)

agent = client.create_agent(
    instructions="""
    Assistant helps with questions on B Lab's certification standards and new requirements. Be brief in your answers.
    Answer ONLY with the facts listed in the sources you retrieve.
    If there isn't enough information, say you don't know. Do not generate answers that don't use the retrieved sources.
    Include source references for each fact you use using square brackets.""",
    
    context_providers=[search_provider]     # Grounding provided as a tool
)

# Minimal run sample with a single question
user_question = "Will the companies that were already certified have to recertify?"
result = await agent.run(
    user_question,
    )

print(result)
```